In [10]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MeshCat as mc
import Distributions
import Random

using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using CSV
using DataFrames


  Activating project at `~/daniel/AiPEX-Projects/warmstarting_NLPs`


In [2]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","cartpole_animation.jl"))

animate_cartpole (generic function with 1 method)

In [3]:
# cartpole 
function dynamics(params::NamedTuple, x::Vector, u)
    # cartpole ODE, parametrized by params. 

    # cartpole physical parameters 
    mc, mp, l = params.mc, params.mp, params.l
    g = 9.81
    
    q = x[1:2]
    qd = x[3:4]

    s = sin(q[2])
    c = cos(q[2])

    H = [mc+mp mp*l*c; mp*l*c mp*l^2]
    C = [0 -mp*qd[2]*l*s; 0 0]
    G = [0, mp*g*l*s]
    B = [1, 0]

    qdd = -H\(C*qd + G - B*u[1])
    xdot = [qd;qdd]
    return xdot 

end

function hermite_simpson(params::NamedTuple, x1::Vector, x2::Vector, u, dt::Real)::Vector
    # TODO: input hermite simpson implicit integrator residual 
     x_mid = 0.5(x1 + x2) + (dt/8) * (dynamics(params, x1, u) - dynamics(params, x2, u))
     res = x1 + (dt/6) * (dynamics(params, x1, u) + 4*dynamics(params, x_mid, u) + dynamics(params, x2, u)) - x2
     return res
end

hermite_simpson (generic function with 1 method)

In [4]:
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

function cartpole_cost(params::NamedTuple, Z::Vector)::Real
    idx, N, xg = params.idx, params.N, params.xg
    Q, R, Qf = params.Q, params.R, params.Qf
    
    # TODO: input cartpole LQR cost 
    J = 0 

    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
       
        J += 0.5*(xi-xg)'*Q*(xi-xg) + 0.5*ui'*R*ui
    end
    
    # dont forget terminal cost 
    xN = Z[idx.x[N]]
    J += 0.5*(xN-xg)'*Qf*(xN-xg)
    return J 
end

function cartpole_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    
    # TODO: create dynamics constraints using hermite simpson 

    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        # TODO: hermite simpson 
        c[idx.c[i]] = hermite_simpson(params, xi, xip1, ui, dt)
    end
    return c 
end

function cartpole_equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    
    
    # TODO: return all of the equality constraints 

    
    return [Z[idx.x[1]] - xic; Z[idx.x[end]] - xg; cartpole_dynamics_constraints(params, Z)] 
end

function solve_cartpole_swingup(σ; verbose=true)
    
    # problem size 
    nx = 4 
    nu = 1 
    dt = 0.05
    tf = 2.0 
    t_vec = 0:dt:tf 
    N = length(t_vec)
    
    # LQR cost 
    Q = diagm(ones(nx))
    R = 0.1*diagm(ones(nu))
    Qf = 10*diagm(ones(nx))
    
    # indexing 
    idx = create_idx(nx,nu,N)
    
    # initial and goal states 
    # xic = [0, 0, 0, 0]
    xic = [σ[1], σ[2], 0, 0]
    xg = [0, pi, 0, 0]
    
    # load all useful things into params 
    params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx,mc = 1.0, mp = 0.2, l = 0.5)
    
    # TODO: primal bounds 
    x_l = fill(-Inf, idx.nz)
    x_u = fill(Inf, idx.nz)
    
    for i = 1:(N-1)
        x_l[idx.u[i]] .= -10
        x_u[idx.u[i]] .= 10
    end

    
    # inequality constraint bounds (this is what we do when we have no inequality constraints)
    c_l = zeros(0)
    c_u = zeros(0)
    function inequality_constraint(params, Z)
        return zeros(eltype(Z), 0)
    end
    
    # initial guess 
    z0 = 0.001*randn(idx.nz)
    
    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    diff_type = :auto 
#     diff_type = :finite
    
    # @show cartpole_equality_constraint(params, z0)
    # @show cartpole_dynamics_constraints(params, z0)
    # @show inequality_constraint(params, z0)
    # @show cartpole_cost(params, z0)
        
    Z, obj, solve_time_sec, term_status = fmincon(cartpole_cost,cartpole_equality_constraint,inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = verbose)
    # term_status = 0
\



    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    
    return X, U, obj, solve_time_sec, term_status, t_vec, params 
end


    


solve_cartpole_swingup (generic function with 1 method)

In [5]:
# X, U, obj, t_vec, params = solve_cartpole_swingup(verbose=true)

# # --------------testing------------------

# Xm = hcat(X...)
# Um = hcat(U...)

# # --------------plotting-----------------
# display(plot(t_vec, Xm', label = ["p" "θ" "ṗ" "θ̇"], xlabel = "time (s)", title = "State Trajectory"))
# display(plot(t_vec[1:end-1],Um',label="",xlabel = "time (s)", ylabel = "u",title = "Controls"))

# # display(animate_cartpole(X, 0.05))



## Solve the DIRCOL NLP for a parameter set

In [6]:
# ## Define upper and lower bounds of the parameters for the Paramaetric Optimal Control Problem
# # using the xic of the cartpole
# using Random, Distributions, CSV, DataFrames
# Random.seed!(123)


# N = 10000 # number of samples
# σ_lower = [0.0, 0.0]
# σ_upper = [0.2, pi/2]

# # Randomly sample the iid parameters uniformly from the given bounds
# d = Product(Uniform.(σ_lower, σ_upper))
# σ_samples = rand(d, N)
# σ_samples = eachcol(σ_samples)

# # Solve the NLP for the parameter sample set
# df = DataFrame(params = Vector{Vector{Float64}}(), X=Vector{Vector{Vector{Float64}}}(), U=Vector{Vector{Vector{Float64}}}(), obj = Float64[], solve_time_sec = Float64[], term_status = MOI.TerminationStatusCode[])

# i = 1
# for σ in σ_samples
#     println("Sample: ", i)
#     println("------------------")
#     X, U, obj, solve_time_sec, term_status, t_vec, params = solve_cartpole_swingup(σ, verbose=false)
#     println("σ: ", σ)
#     println("Objective Value: ", obj)
#     println("TerminationStatusCode: ", term_status)
#     println("")

#     push!(df, [σ, X, U, obj, solve_time_sec, term_status])
#     i += 1
# end



In [7]:

# CSV.write("data/cartpole_DIRCOL_10000.csv", df)

df

UndefVarError: UndefVarError: `df` not defined

## Load in the Warmstarts and Solve for the Refined Trajectories

In [32]:
using CSV

df = DataFrame(CSV.File("data/warmstart_cartpole.csv"))
X_warmstart_str = df.X_warmstart[1]

# Function to convert the string representation to a 2D array
function convert_to_2d_array(str::String)
    # Remove the brackets and newline characters
    clean_str = replace(str, r"[\[\]\n]" => "")
    # Split the string into individual numbers
    num_strs = split(clean_str)
    # Convert the numbers to Float64
    nums = parse.(Float64, num_strs)
    # Reshape the flat array into a 2D array
    num_rows = count(x -> x == '\n', str) + 1
    num_cols = length(nums) ÷ num_rows
    return reshape(nums, num_cols, num_rows)'
end

X_warmstart = convert_to_2d_array(X_warmstart_str)
X_warmstart = X_warmstart'
X_warmstart = [X_warmstart[:, i] for i in 1:size(X_warmstart, 2)]
@show size(X_warmstart[1])

size(X_warmstart[1]) = (4,)


(4,)